In [25]:

import pandas as pd
import os 
from utility import get_friendly_name

ground_truth_file = r'D:\images\tnia-python-images\imagesc\2025_06_02_check_ABRF_labels\ground_truth_coords.csv'
coordinate_data_file = r'D:\images\tnia-python-images\imagesc\2025_06_02_check_ABRF_labels\coordinate_data_deidentified.csv'

base_name = 'out_c00_dr90'

friendly_name = get_friendly_name(f"{base_name}_label.tif")
print(f"Friendly name: {friendly_name}")

Friendly name: nuclei1


In [92]:
ground_truth = pd.read_csv(ground_truth_file)
ground_truth['base_name'] = ground_truth['path'].apply(os.path.basename)
ground_truth.head()

,Unnamed: 0,x,y,z,path,base_name
0,0,5.628644,9.198676,8.751923,../../ground truth/nuclei/out_c00_dr90_label.tif,out_c00_dr90_label.tif
1,1,22.816774,9.033185,6.689963,../../ground truth/nuclei/out_c00_dr90_label.tif,out_c00_dr90_label.tif
2,2,22.618367,26.020343,15.707744,../../ground truth/nuclei/out_c00_dr90_label.tif,out_c00_dr90_label.tif
3,3,13.676880,23.516335,14.263662,../../ground truth/nuclei/out_c00_dr90_label.tif,out_c00_dr90_label.tif
4,4,25.001824,8.219340,21.236781,../../ground truth/nuclei/out_c00_dr90_label.tif,out_c00_dr90_label.tif


In [93]:
data = pd.read_csv(coordinate_data_file)
data = data[data.csv_path.str.contains('nuclei', na=False)]
print(data.shape)
data.head()

(537, 6)


,x,y,z,intensity,volume,csv_path
0,22.803,9.024,4.181,56620971.0,437.896,Q6.13/R_3Rxaf07hES8CPgE_nuclei1.csv
1,5.669,9.164,5.376,63811635.0,482.954,Q6.13/R_3Rxaf07hES8CPgE_nuclei1.csv
2,24.999,8.223,13.036,57406958.0,441.048,Q6.13/R_3Rxaf07hES8CPgE_nuclei1.csv
3,13.722,23.521,8.778,78224458.0,522.320,Q6.13/R_3Rxaf07hES8CPgE_nuclei1.csv
4,22.547,26.006,9.65,69841365.0,473.768,Q6.13/R_3Rxaf07hES8CPgE_nuclei1.csv


In [94]:
response_id = r'R_31bjqd6Mm8wBxN5'
response_id = 'R_3RxOa8kaiyul4bG'
response_id = 'R_1M6DoAmYEY3Jrvm'
response_id = 'R_3lAJ9xY4kGlL99f'

In [95]:
print(base_name)
points = ground_truth[ground_truth['base_name'].str.contains(base_name)][['x', 'y', 'z']].values
print(points)

out_c00_dr90
[[ 5.62864416  9.19867592  8.75192296]
 [22.8167741   9.0331853   6.68996262]
 [22.61836734 26.02034264 15.707744  ]
 [13.67688018 23.51633499 14.26366235]
 [25.00182357  8.21933956 21.2367806 ]]


In [96]:
responses = data[data.csv_path.str.contains(response_id, na=False)]
responses = responses[responses.csv_path.str.contains(friendly_name, na=False)]
responses = responses[['x', 'y', 'z']].values.astype(float)
print('Responses shape:', responses.shape)


Responses shape: (5, 3)


In [97]:
print('ground truth\n', points)
print('response\n', responses)


ground truth
 [[ 5.62864416  9.19867592  8.75192296]
 [22.8167741   9.0331853   6.68996262]
 [22.61836734 26.02034264 15.707744  ]
 [13.67688018 23.51633499 14.26366235]
 [25.00182357  8.21933956 21.2367806 ]]
response
 [[5.88859531 3.47025625 2.16835938]
 [6.48164634 5.67132578 2.35993031]
 [2.25828997 5.69045482 0.99721946]
 [2.06101634 6.27222585 3.21951461]
 [2.25909068 1.44427836 1.33057782]]


In [98]:

import numpy as np
temp=np.rot90(responses[:,:2], k=1, axes=(0, 1))
responses=np.transpose(temp)
print('rotated response\n', responses)


rotated response
 [[3.47025625 5.88859531]
 [5.67132578 6.48164634]
 [5.69045482 2.25828997]
 [6.27222585 2.06101634]
 [1.44427836 2.25909068]]


In [99]:
points_ = points.copy()
response_ = responses.copy()

In [ ]:
points = points_
gt = points
responses = response_*20
td = responses

scale = gt[:, :2].std(axis=0).mean() / td[:, :2].std(axis=0).mean()
td[:, :2] *= scale
translation = np.array([(td[:, :2].mean(axis=0) - gt[:, :2].mean(axis=0))])

print('Scale:', scale)
print('Translation:', translation)

print(td)

Scale: 0.20069910118738982
Translation: [[72.2456664  60.59697891]]
[[ 69.405125   117.77190625]
 [113.42651568 129.63292683]
 [113.80909633  45.1657994 ]
 [125.44451709  41.22032689]
 [ 28.88556727  45.18181358]]


In [139]:
from pycpd import RigidRegistration
registration_params = {
    'max_iterations': 10000,
}

In [145]:
reg = RigidRegistration(
                X=points[:, :2],
                Y=responses[:, :2],
                #s=scale,
                #t=translation,
                **registration_params,
            )

test_coords_tf, test_tf = reg.register()



In [148]:
help(RigidRegistration)

Help on class RigidRegistration in module pycpd.rigid_registration:

class RigidRegistration(pycpd.emregistration.EMRegistration)
 |  RigidRegistration(R=None, t=None, s=None, *args, **kwargs)
 |  
 |  Rigid registration.
 |  
 |  Attributes
 |  ----------
 |  R: numpy array (semi-positive definite)
 |      DxD rotation matrix. Any well behaved matrix will do,
 |      since the next estimate is a rotation matrix.
 |  
 |  t: numpy array
 |      1xD initial translation vector.
 |  
 |  s: float (positive)
 |      scaling parameter.
 |  
 |  A: numpy array
 |      Utility array used to calculate the rotation matrix.
 |      Defined in Fig. 2 of https://arxiv.org/pdf/0905.2635.pdf.
 |  
 |  YPY: float
 |      Denominator value used to update the scale factor.
 |      Defined in Fig. 2 and Eq. 8 of https://arxiv.org/pdf/0905.2635.pdf.
 |  
 |  X_hat: numpy array
 |      Centered target point cloud.
 |      Defined in Fig. 2 of https://arxiv.org/pdf/0905.2635.pdf.
 |  
 |  Method resolution

In [146]:
test_coords_tf

array([[17.73196387, 15.62058374],
       [18.17693628, 15.74681472],
       [18.19215653, 14.89000909],
       [18.31071899, 14.85154716],
       [17.33066753, 14.87877102]])

In [42]:
test_tf

(1.0232684916840524,
 array([[ 0.8048173 , -0.59352263],
        [ 0.59352263,  0.8048173 ]]),
 array([-3.90553282, 10.83651613]))

In [43]:
points[:,:2]

array([[ 5.62864416,  9.19867592],
       [22.8167741 ,  9.0331853 ],
       [22.61836734, 26.02034264],
       [13.67688018, 23.51633499],
       [25.00182357,  8.21933956]])

In [46]:
import numpy as np
temp=np.rot90(points[:,:2], k=1, axes=(0, 1))
np.transpose(temp)

array([[ 9.19867592,  5.62864416],
       [ 9.0331853 , 22.8167741 ],
       [26.02034264, 22.61836734],
       [23.51633499, 13.67688018],
       [ 8.21933956, 25.00182357]])

In [47]:
(np.array([[0.0, 1.0], [-1.0, 0.0]]) @ points[:, :2].T).T

array([[  9.19867592,  -5.62864416],
       [  9.0331853 , -22.8167741 ],
       [ 26.02034264, -22.61836734],
       [ 23.51633499, -13.67688018],
       [  8.21933956, -25.00182357]])

In [44]:
help(RigidRegistration)

Help on class RigidRegistration in module pycpd.rigid_registration:

class RigidRegistration(pycpd.emregistration.EMRegistration)
 |  RigidRegistration(R=None, t=None, s=None, *args, **kwargs)
 |  
 |  Rigid registration.
 |  
 |  Attributes
 |  ----------
 |  R: numpy array (semi-positive definite)
 |      DxD rotation matrix. Any well behaved matrix will do,
 |      since the next estimate is a rotation matrix.
 |  
 |  t: numpy array
 |      1xD initial translation vector.
 |  
 |  s: float (positive)
 |      scaling parameter.
 |  
 |  A: numpy array
 |      Utility array used to calculate the rotation matrix.
 |      Defined in Fig. 2 of https://arxiv.org/pdf/0905.2635.pdf.
 |  
 |  YPY: float
 |      Denominator value used to update the scale factor.
 |      Defined in Fig. 2 and Eq. 8 of https://arxiv.org/pdf/0905.2635.pdf.
 |  
 |  X_hat: numpy array
 |      Centered target point cloud.
 |      Defined in Fig. 2 of https://arxiv.org/pdf/0905.2635.pdf.
 |  
 |  Method resolution